In [1]:
import numpy as np
import pandas as pd
import os
import sys
import base64
import unicodedata
import codecs
from chardet.universaldetector import UniversalDetector

from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.util import ngrams
from scipy.sparse import lil_matrix

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Задание

## Определим языки

In [2]:
sourceFileName = "2.tmp"
with codecs.open(sourceFileName, "r") as file:
    for line in file.readlines():
        print(line)

Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第二十一歲

                                        生日，霍比頓人的話語和興奮也很多。 ＃＃＃＃Bilbo非常富有，非常奇特，自從他的驚人的失踪和意想不

            到的回歸以來，一直是希雷六十年的奇蹟。他從旅行中帶回來的財富現在已經成為一個地方的傳說，無論老民間說

什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力來驚嘆。

              時間穿了，但似乎對Baggins先生影響不大。在九十年代，他和五十歲時一樣。在九十九歲的時候，他們開

              始把他稱得上是保存完好的，但是不會改變的。有一些人搖頭，認為這太好了，任何人都應該擁有（顯然）永久的

青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，麻煩會來

  的！”＃＃但是到目前為止，麻煩沒有到來。正如巴金斯先生慷慨解囊，大多數人都願意赦免他的遺體和幸運。他

仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多忠實的崇

      拜者。但他沒有親密的朋友，直到他的一些年輕的表親才開始長大。 ＃＃這些最老的，Bilbo的最愛，是年

            輕的Frodo Baggins。當比爾博九十九歲的時候，他把弗洛多當作他的繼承人，把他帶到了袋末，薩

                          克維爾 - 巴金斯的希望終於被打破了。比爾博和佛羅多恰好有同樣的生日，9月22日。 “你最好過來住這

              裡，弗洛多我的小伙子，”比爾博說， “然後我們可以一起慶祝我們的生日聚會。”當時，弗羅多仍然在他的補

  間，因為愛好者們把三十三歲的童年和未來之間的不負責任的二十年代稱為。 ＃＃十二年過去了。 Baggi

              nses每年都在Bag End舉行了非常熱鬧的生日聚會;但是現在據了解，那個秋天正在計劃一些非常特別

                        的事情。比爾博將是一個111，一個相當好奇的數字和一個非常受人尊敬的霍比特人的年

Это китайский

In [3]:
sourceFileName = "3.tmp"
with codecs.open(sourceFileName, "r") as file:
    for line in file.readlines():
        print(line)

When Mr. Bilbo Baggins of Bag End announced that h

            e would shortly be celebrating his eleventy-first 

            birthday with a party of special magnificence, the

            re was much talk and excitement in Hobbiton.

 

 

 

            

 Bilbo was very rich and very peculiar, and had b

            een the wonder of the Shire for sixty years, ever 

            since his remarkable disappearance and unexpected 

            return. The riches he had brought back from his tr

            avels had now become a local legend, and it was po

            pularly believed, whatever the old folk might say,

             that the Hill at Bag End was full of tunnels stuf

            fed with treasure. And if that was not enough for 

            fame, there was also his prolonged vigour to marve

            l at. Time wore on, but it seemed to have little e

            ffect on Mr. Baggins. At ninety he was much the sa

            me as at fifty. At ninety-nine they 


            ook and Miss Melilot Brandybuck got on a table and

             with bells in their hands began to dance the Spri

            ngle-ring: a pretty dance, but rather vigorous.

 



             But Bilbo had not finished. Seizing a horn from a

             youngster near by, he blew three loud hoots. The 

            noise subsided. Ishall not keep you long, he cried

            . Cheers from all the assembly. Ihave called you a

            ll together for a Purpose. Something in the way th

            at he said this made an impression. There was almo

            st silence, and one or two of the Tooks pricked up

             their ears.

 

 Indeed, for Three Purposes! First 

            of all, to tell you that I am immensely fond of yo

            u all, and that eleventy-one years is too short a 

            time to live among such excellent and admirable ho

            bbits. Tremendous outburst of approval.

 

 I don’t

           know half of you half 

             of mortal race who possessed it. It would possess

             him.

 

 ‘In Eregion long ago many Elven-rings wer

          e made, magic rings as you call them, and they wer

            e, of course, of various kinds: some more potent a

            nd some less. The lesser rings were only essays in

             the craft before it was full-grown, and to the El

            ven-smiths they were but trifles – yet still to my

           mind dangerous for mortals. But the Great Rings, 

            the Rings of Power, they were perilous.

 

 ‘A mort

          al, Frodo, who keeps one of the Great Rings, does 

            not die, but he does not grow or obtain more life,

             he merely continues, until at last every minute i

            s a weariness. And if he often uses the Ring to ma

            ke himself invisible, he fades: he becomes in the 

            end invisible permanently, and walks in the twilig

            ht under the eye of the dark p


 After lunch, th

            e Sackville-Bagginses, Lobelia and her sandy-haire

            d son, Lotho, turned up, much to Frodo’s annoyance

          . ‘Ours at last!’ said Lobelia, as she stepped ins

        ide. It was not polite; nor strictly true, for the

             sale of Bag End did not take effect until midnigh

            t. But Lobelia can perhaps be forgiven: she had be

            en obliged to wait about seventy-seven years longe

            r for Bag End than she once hoped, and she was now

             a hundred years old. Anyway, she had come to see 

            that nothing she had paid for had been carried off

            ; and she wanted the keys. It took a long while to

             satisfy her, as she had brought a complete invent

            ory with her and went right through it. In the end

             she departed with Lotho and the spare key and the

             promise that the other key would be left at the G

            amgees’ in Bags

            ugh fast turning yellow were still thick, and the 

            ground at its feel was fairly dry and sheltered. W

            hen they came to make their meal, they found that 

            the Elves had filled their bottles with a clear dr

            ink, pale golden in colour: it had the scent of a 

            honey made of many flowers, and was wonderfully re

            freshing. Very soon they were laughing, and snappi

            ng their fingers at rain, and at Black Riders. The

             last few miles, they felt, would soon be behind t

            hem.

 

 Frodo propped his back against the tree-tr

            unk, and closed his eyes. Sam and Pippin sat near,

             and they began to hum, and then to sing softly:

 

            

 Ho! Ho! Ho! to the bottle I go

 

 To heal my hea

            rt and drown my woe.

 

 Rain may fall and wind may

             blow,

 

 And many miles be still to go,

 

 But un

            der a tall tree 

Это английский

In [4]:
sourceFileName = "4.tmp"
with codecs.open(sourceFileName, "r") as file:
    for line in file.readlines():
        print(line)

Bag EndのBilbo Baggins氏は、18歳の誕生日を特別な壮大なパーティでまもなく祝うと

                                            発表したとき、ホビットンには多くの話と興奮がありました。ビルボは非常に豊かで非常に独特で、彼の顕著な

失踪と予期せぬ復帰以来、60年にわたりシャイアの驚異であった。彼が旅行から持ち帰った富は今や地元の伝

    説になっていて、昔の人々が言うように、バッグエンドの丘は宝物で詰まったトンネルでいっぱいだったと一般

に信じられていました。それが名声のために十分でないならば、彼の長年の活力も驚くべきものでした。時間は

かかりましたが、バギンズ氏にはほとんど影響がないようでした。 90歳で彼は50歳とほとんど同じだった

          。 99歳で彼は彼をよく保存されていると呼ぶようになったが、変わらずに近くになっただろう。頭を振って

      、これがあまりにも良いことだと思った人もいました。誰も（明らかに）永遠の青年と（評判が良い）無尽蔵の

富を所有すべきであるということは不公平に思えました。 "それは支払われなければならないだろう"と彼ら

      は言った。 'それは自然ではなく、問題はそれから来る！'＃＃しかし、これまでのトラブルは来ていません

      でした。 Baggins氏は彼のお金で寛大であったので、ほとんどの人が彼の奇妙さと彼の幸運を許してく

                れた。彼は親戚（もちろんSackville-Bagginsesを除く）と一緒に訪問していましたが、貧

                                      困層や重要でない家族のホビットには多くの献身者がいました。しかし、若い従兄弟たちが成長し始めるまで、

彼は親しい友人がいませんでした。 ##これらの長男とビルボのお気に入りは、若いフロド・バギンズだった

      。ビルボが99歳のとき、彼は彼の相続人としてフロドを採用し、彼をバッグエンドに住まわせた。サックビル

    ・バギンゼスの希望はついに破れた。ビルボとフロドは9月22日に同じ誕生日を迎えました。 「あなたはこ

        こでもっと来て、ここに住んでいました、私

                                        Merry Brandybuck（彼の本名はMeriadocでしたが、めったに思い出せませんでした）

                                                でした。フロドはシャイアのいたるところで彼らと一緒に踏みにじった。しかし、より頻繁に彼は自分自身でさ

まよっていたし、賢明な人々の驚きに、彼は時々、星空の下で丘や木々を歩いて家から遠くに見えた。メリーと

ピッピンは、ビルボがしたように、エルフを時々訪ねたと疑った。 ##時間が経つにつれて、人々はフロドも

      良い「守り」の兆候を示したことに気づき始めた。外見上、彼はちょうどトゥイーンの中から丈夫で元気なホビ

ットの姿を保った。 「一部の人々はすべての運を持っている」と彼らは言った。フロドがいつももっと冷静な

  50歳に近づくまでは、それは奇妙だと思うようになりました。最初のショックの後、＃Frodo自身は、彼

              自身の主人であり、Bag EndのMr. Bagginsはむしろ楽しいと分かった。何年もの間、彼はか

                                    なり幸せだったし、将来についてはあまり心配しなかった。しかし、彼がビルボと一緒に行かなかったことの後

悔は自分自身には未知の半分が着実に成長していた。彼は自分自身が、特に秋に、野生の土地について、そして

今までに見たことのない奇妙な山々の幻想が自分の夢の中に入ったときに、いつか不思議に思っていました。彼

は自分自身に言うようになった：「おそらく私はある日、自分自身の川を渡るだろう」彼の心の他の半分はいつ

も「まだではない」と答えた。彼の50代の誕生日は近づいていた：50は何とか有意義だったと感じた数だっ

        た。いずれにせよ、その頃には冒険が突然Bilboに起こった。フロドは落ち着きがなくなり、古い道はあま

          りにもよく喧嘩していたようだ。彼は地図を見て、その縁を越えて何が敷かれているのか疑問に思った。シャイ

アで作られた地図は、その国境を越えてほとんど白い空間を示した。彼は遠く離れてさらに遠くをさまよった。

メリーと

Это японский

## Классификатор

### Считываем данные

In [5]:
data = []
file_names = ["2.tmp", "3.tmp", "4.tmp"]

for file_name in file_names:
    with codecs.open(file_name, "r") as file:
        for line in file.readlines():
            data.append((line.strip('\x00').strip(), file_name[0]))

In [6]:
len(data)

8465

In [7]:
data[:5]

[('Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第二十一歲', '2'),
 ('生日，霍比頓人的話語和興奮也很多。 ＃＃＃＃Bilbo非常富有，非常奇特，自從他的驚人的失踪和意想不', '2'),
 ('到的回歸以來，一直是希雷六十年的奇蹟。他從旅行中帶回來的財富現在已經成為一個地方的傳說，無論老民間說', '2'),
 ('什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力來驚嘆。', '2'),
 ('時間穿了，但似乎對Baggins先生影響不大。在九十年代，他和五十歲時一樣。在九十九歲的時候，他們開', '2')]

In [8]:
data[3000:3005]

[('inish now? You say the ring is dangerous, far more', '3'),
 ('dangerous than I guess. In what way?’', '3'),
 ('', '3'),
 ('‘In many', '3'),
 ('ways,’ answered the wizard. It is far more powerf', '3')]

In [9]:
data[7000:7005]

[('をもう一度言うなら、私はしなければならない。それから、あなたは灰色のガンダルフが覆されていないのを見', '4'),
 ('るでしょう。彼はホビットに足を踏み入れ、彼は背が高くて脅かされるようでした。彼の影は小さな部屋に満ち', '4'),
 ('ていた。 ##ビルボは、彼の手が彼のポケットにかがんで、ハード呼吸、壁に戻った。彼らは互いに向き合っ', '4'),
 ('て立ち、部屋の空気が混じっていた。ガンダルフの目はホビットの上で曲がっていた。ゆっくりと手が緩み、彼', '4'),
 ('は震え始めた。 ##あなたが何が来たのかわからない、ガンダルフ。あなたはこれまでにこれほど似ていたこ', '4')]

In [10]:
frame = pd.DataFrame(data, columns=['Line', 'Flag'])

In [11]:
frame.head()

,Line,Flag
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...,2
1,生日，霍比頓人的話語和興奮也很多。 ＃＃＃＃Bilbo非常富有，非常奇特，自從他的驚人的失踪...,2
2,到的回歸以來，一直是希雷六十年的奇蹟。他從旅行中帶回來的財富現在已經成為一個地方的傳說，無論...,2
3,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...,2
4,時間穿了，但似乎對Baggins先生影響不大。在九十年代，他和五十歲時一樣。在九十九歲的時候...,2


In [12]:
frame["Line"]

0       Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...
1       生日，霍比頓人的話語和興奮也很多。 ＃＃＃＃Bilbo非常富有，非常奇特，自從他的驚人的失踪...
2       到的回歸以來，一直是希雷六十年的奇蹟。他從旅行中帶回來的財富現在已經成為一個地方的傳說，無論...
3       什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...
4       時間穿了，但似乎對Baggins先生影響不大。在九十年代，他和五十歲時一樣。在九十九歲的時候...
5       始把他稱得上是保存完好的，但是不會改變的。有一些人搖頭，認為這太好了，任何人都應該擁有（顯然...
6       青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，...
7       的！”＃＃但是到目前為止，麻煩沒有到來。正如巴金斯先生慷慨解囊，大多數人都願意赦免他的遺體和...
8       仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多...
9       拜者。但他沒有親密的朋友，直到他的一些年輕的表親才開始長大。 ＃＃這些最老的，Bilbo的最...
10      輕的Frodo Baggins。當比爾博九十九歲的時候，他把弗洛多當作他的繼承人，把他帶到了...
11      克維爾 - 巴金斯的希望終於被打破了。比爾博和佛羅多恰好有同樣的生日，9月22日。 “你最好...
12      裡，弗洛多我的小伙子，”比爾博說， “然後我們可以一起慶祝我們的生日聚會。”當時，弗羅多仍然...
13      間，因為愛好者們把三十三歲的童年和未來之間的不負責任的二十年代稱為。 ＃＃十二年過去了。 B...
14      nses每年都在Bag End舉行了非常熱鬧的生日聚會;但是現在據了解，那個秋天正在計劃一些...
15      的事情。比爾博將是一個111，一個相當好奇的數字和一個非常受人尊敬的霍比特人的年齡（老托克自...
16      到130）;弗羅多將要三十三，三十三歲）一個重要的數字：他的年齡的日期。 ＃＃舌頭開始在Ho...
17      on和Byw

### Подготовка признаков

In [13]:
[x for x in ngrams('sentence word', 3)]

[('s', 'e', 'n'),
 ('e', 'n', 't'),
 ('n', 't', 'e'),
 ('t', 'e', 'n'),
 ('e', 'n', 'c'),
 ('n', 'c', 'e'),
 ('c', 'e', ' '),
 ('e', ' ', 'w'),
 (' ', 'w', 'o'),
 ('w', 'o', 'r'),
 ('o', 'r', 'd')]

In [14]:
MAX_N = 3
lm = {n:set() for n in range(1, MAX_N)}
def extract_n_grams(line):
    for n in range(1, MAX_N):
        ngram = ngrams(line, n)
        for item in ngram:
            lm[n].add(item)

for line in frame["Line"]:
        extract_n_grams(line)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [15]:
lm[1]

{('種',),
 ('擋',),
 ('池',),
 ('絡',),
 ('敞',),
 ('ー',),
 ('僵',),
 ('悶',),
 ('導',),
 ('R',),
 ('害',),
 ('朗',),
 ('直',),
 ('飄',),
 ('系',),
 ('牢',),
 ('跳',),
 ('廳',),
 ('港',),
 ('駄',),
 ('冷',),
 ('具',),
 ('腦',),
 ('岸',),
 ('增',),
 ('股',),
 ('紗',),
 ('牧',),
 ('愧',),
 ('沖',),
 ('臉',),
 ('試',),
 ('踢',),
 ('站',),
 ('院',),
 ('狹',),
 ('狐',),
 ('隣',),
 ('朵',),
 ('州',),
 ('渠',),
 ('カ',),
 ('沸',),
 ('別',),
 ('震',),
 ('情',),
 ('奈',),
 ('次',),
 ('寸',),
 ('態',),
 ('読',),
 ('略',),
 ('期',),
 ('詰',),
 ('適',),
 ('沢',),
 ('嘩',),
 ('做',),
 ('ん',),
 ('見',),
 ('リ',),
 ('戰',),
 ('第',),
 ('請',),
 ('保',),
 ('獰',),
 ('割',),
 ('植',),
 ('吐',),
 ('ざ',),
 ('択',),
 ('六',),
 ('矮',),
 ('笑',),
 ('慣',),
 ('護',),
 ('按',),
 ('瘦',),
 ('鋸',),
 ('窓',),
 ('隧',),
 ('成',),
 ('雖',),
 ('華',),
 ('極',),
 ('豊',),
 ('棄',),
 ('0',),
 ('仍',),
 ('址',),
 ('三',),
 ('紙',),
 ('緑',),
 ('ダ',),
 ('奇',),
 ('遭',),
 ('疊',),
 ('饒',),
 ('褪',),
 ('幹',),
 ('3',),
 ('拾',),
 ('巴',),
 ('咬',),
 ('蟾',),
 ('館',),
 ('籍',),
 ('手',),
 ('悉',),
 ('！',),
 ('孔',),
 

In [16]:
features = []
for n in lm.keys():
    features.extend(lm[n])
features = np.array(features)

In [17]:
features

array([('種',), ('擋',), ('池',), ..., ('日', 'で'), ('幹', '上'), ('和', '閃')], dtype=object)

In [18]:
features_dict = {features[i]: i for i in range(len(features))}

In [19]:
features_dict

{('種',): 0,
 ('擋',): 1,
 ('池',): 2,
 ('絡',): 3,
 ('敞',): 4,
 ('ー',): 5,
 ('僵',): 6,
 ('悶',): 7,
 ('導',): 8,
 ('R',): 9,
 ('害',): 10,
 ('朗',): 11,
 ('直',): 12,
 ('飄',): 13,
 ('系',): 14,
 ('牢',): 15,
 ('跳',): 16,
 ('廳',): 17,
 ('港',): 18,
 ('駄',): 19,
 ('冷',): 20,
 ('具',): 21,
 ('腦',): 22,
 ('岸',): 23,
 ('增',): 24,
 ('股',): 25,
 ('紗',): 26,
 ('牧',): 27,
 ('愧',): 28,
 ('沖',): 29,
 ('臉',): 30,
 ('試',): 31,
 ('踢',): 32,
 ('站',): 33,
 ('院',): 34,
 ('狹',): 35,
 ('狐',): 36,
 ('隣',): 37,
 ('朵',): 38,
 ('州',): 39,
 ('渠',): 40,
 ('カ',): 41,
 ('沸',): 42,
 ('別',): 43,
 ('震',): 44,
 ('情',): 45,
 ('奈',): 46,
 ('次',): 47,
 ('寸',): 48,
 ('態',): 49,
 ('読',): 50,
 ('略',): 51,
 ('期',): 52,
 ('詰',): 53,
 ('適',): 54,
 ('沢',): 55,
 ('嘩',): 56,
 ('做',): 57,
 ('ん',): 58,
 ('見',): 59,
 ('リ',): 60,
 ('戰',): 61,
 ('第',): 62,
 ('請',): 63,
 ('保',): 64,
 ('獰',): 65,
 ('割',): 66,
 ('植',): 67,
 ('吐',): 68,
 ('ざ',): 69,
 ('択',): 70,
 ('六',): 71,
 ('矮',): 72,
 ('笑',): 73,
 ('慣',): 74,
 ('護',): 75,
 ('按',): 76,
 ('瘦',): 

### Создание разреженной tf-idf таблицы признаков

In [20]:
class Vectorizer:
    @staticmethod
    def count_fit_transform(data, features, features_dict, lm):
        '''
        Params
        ------
        data : array of words after cut of not surnames
        
        Returns
        -------
        X : scipy.sparse matrix, shape = (n_samples, n_features)
            Document-term matrix.
        '''
        X = lil_matrix((data.size, features.size))
        for i in range(data.size):
            for n in lm.keys():
                for gr in ngrams(data[i], n):
                    X[i, features_dict[gr]] += 1
        return X
              
    @staticmethod
    def tfidf_transform(X):
        '''
        Params
        ------
        X : sparse matrix after count_fit_transform
        
        Returns
        -------
        X : scipy.sparse matrix, shape = (n_samples, n_features)
            Document-term tfidf matrix.
        '''
        print(1)
        # amount of nonzero terms in each colomn
        c_nonzero = [X[:, col].count_nonzero() for col in range(X.shape[1])]
        print(2)
        # sums of amounts of terms in each row
        r_sum = [x.item((0, 0)) for x in X.sum(axis=1)]
        print(3)
        
        doc_size = float(X.shape[0])
        
        print(4)
        # tfidf
        for i, j in X.nonzero():
            X[i, j] = X[i, j]*np.log(doc_size/c_nonzero[j]) / r_sum[i]
            
        print(5)
        return X

### Подготовка обучающих данных

In [21]:
%time X = Vectorizer.count_fit_transform(frame["Line"], features, features_dict, lm)
#%time X = Vectorizer.tfidf_transform(X)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: generator 'ngrams' raised StopIteration


CPU times: user 6.87 s, sys: 113 ms, total: 6.98 s
Wall time: 7.08 s


In [22]:
X.shape

(8465, 34315)

### Обучение модели

In [23]:
%%time
Y = frame["Flag"]
train_score = []
test_score = []

for train_indices, test_indices in KFold(Y.size, n_folds=10, shuffle=True):
    model = LogisticRegression()
    model.fit(X[train_indices], Y[train_indices])
    train_score.append(float(model.score(X[train_indices], Y[train_indices])))
    test_score.append(float(model.score(X[test_indices], Y[test_indices])))

CPU times: user 4.6 s, sys: 199 ms, total: 4.8 s
Wall time: 4.83 s


In [24]:
print("Train score: ", np.array(train_score).mean())
print("Test score: ", np.array(test_score).mean())

Train score:  0.999763731294
Test score:  0.999645529626
